In [1]:
fo=open("hindi_paragraph.txt","r")
fo_text=fo.read()
fo.close()

In [2]:
sentences=fo_text.split(".")

In [3]:
len(sentences)

154

In [4]:
import re
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
def ngram_to_list(sentences):
    corpus=[]
    count_is=0
    for i in sentences:
        review=re.sub("[^a-zA-Z]"," ",i)
        corpus.append(review)
        if len(i)>count_is:
            count_is=len(i)
    cv=CountVectorizer(binary=True,ngram_range=(1,count_is))
    X=cv.fit_transform(corpus)
    list_ngrams=list(cv.vocabulary_)
    return list_ngrams


In [6]:
corpus_of_hindi_text=ngram_to_list(sentences)

In [7]:
len(corpus_of_hindi_text)

30818

In [8]:
import pandas as pd

# Creating dataframe of hindi text

In [9]:
df_hindi={}
df_hindi["Response"]=corpus_of_hindi_text
df_hindi["Label"]=[1 for i in range(len(corpus_of_hindi_text))]
df_hindi=pd.DataFrame(df_hindi)

In [10]:
print(df_hindi.shape)
print("total no of unique values are :")
print(df_hindi.nunique())
df_hindi.head()

(30818, 2)
total no of unique values are :
Response    30818
Label           1
dtype: int64


,Response,Label
0,paisa,1
1,fek,1
2,tamasaha,1
3,dekh,1
4,usse,1


# creating dataframe of english text

In [11]:
fo=open("chatbot.txt","r")
chatbot_data=fo.read()
fo.close()

In [12]:
list_of_sentences=chatbot_data.replace("\t"," ").split("\n")

In [13]:
corpus_of_normal_text=ngram_to_list(list_of_sentences)
print(len(corpus_of_normal_text))

90834


# Creating dataframe of normal text¶

In [14]:
df_chat={}
df_chat["Response"]=corpus_of_normal_text
df_chat["Label"]=[0 for i in range(len(corpus_of_normal_text))]
df_chat=pd.DataFrame(df_chat)

In [15]:
print(df_chat.shape)
print("total no of unique values are :")
print(df_chat.nunique())
df_chat.head()

(90834, 2)
total no of unique values are :
Response    90834
Label           1
dtype: int64


,Response,Label
0,what,0
1,are,0
2,your,0
3,interests,0
4,am,0


# combining df_hindi and df_chat

In [16]:
df_main=pd.concat([df_hindi,df_chat],ignore_index=True)

In [17]:
print(df_main.shape)
print("total no of unique values are :")
print(df_main.nunique())
df_main.head()

(121652, 2)
total no of unique values are :
Response    121575
Label            2
dtype: int64


,Response,Label
0,paisa,1
1,fek,1
2,tamasaha,1
3,dekh,1
4,usse,1


In [18]:
df_main.tail()

,Response,Label
121647,use sense similar to sonar to determine the lo...,0
121648,sense similar to sonar to determine the locati...,0
121649,dolphins use sense similar to sonar to determi...,0
121650,use sense similar to sonar to determine the lo...,0
121651,dolphins use sense similar to sonar to determi...,0


# converting of text to tokens

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [20]:
vocab=100000

In [21]:
sentences=[]
for i in list(df_chat["Response"]):
    review=re.sub("[^a-zA-Z]"," ",i)
    review=review.split()
    review=[i for i in review if i not in sentences and len(i)>1]
    sentences.extend(review)
len(sentences)

1854

In [22]:
fo=open("paragraph.txt","r")
fo_text=fo.read()
fo.close()

In [23]:
for i in fo_text.split():
    i=re.sub("[^a-zA-Z]","",i)
    if i not in sentences and len(i)>1:
        sentences.append(i)
len(sentences)

3920

In [24]:
tokenizer=Tokenizer(num_words=vocab,oov_token="<OOV>")

In [25]:
tokenizer.fit_on_texts(sentences)

In [26]:
len(tokenizer.word_index)

3618

In [27]:
sequences=tokenizer.texts_to_sequences(df_main["Response"])

In [28]:
max_len=0
for i in sequences:
    if len(i)>max_len:
        max_len=len(i)
print(max_len)

69


In [29]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [30]:
padded=pad_sequences(sequences,maxlen=max_len)

In [31]:
padded

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    0,    1],
       ...,
       [   0,    0,    0, ...,    3, 1956, 1957],
       [   0,    0,    0, ..., 1956, 1957, 1958],
       [   0,    0,    0, ..., 1956, 1957, 1958]])

# splitting training and test data

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(padded, df_main["Label"], test_size=0.40,random_state=0)

In [34]:
print("the shape of X_train is : ",X_train.shape)
print("the shape of X_test is : ",X_test.shape)
print("the shape of y_train is : ",y_train.shape)
print("the shape of y_test is : ",y_test.shape)

the shape of X_train is :  (72991, 69)
the shape of X_test is :  (48661, 69)
the shape of y_train is :  (72991,)
the shape of y_test is :  (48661,)


In [35]:
print(y_test.value_counts())

0    36386
1    12275
Name: Label, dtype: int64


In [36]:
import keras

In [37]:
model = keras.models.Sequential([
    keras.layers.Embedding(vocab, 64),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(rate=0.25),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dropout(rate=0.25),
    keras.layers.Dense(1, activation="sigmoid")
])

In [38]:
model.compile(optimizer="adam",loss='binary_crossentropy',metrics=["acc"])

In [39]:
history = model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test),  verbose=1)

2281/2281 [==============================] - 402s 170ms/step - loss: 0.0380 - acc: 0.9856 - val_loss: 0.0073 - val_acc: 0.9982


In [40]:
y_predict=model.predict(X_test)

1521/1521 [==============================] - 3s 1ms/step


In [41]:
y_predict=y_predict>0.5

In [42]:
from sklearn.metrics import accuracy_score,classification_report

In [43]:
accuracy_score(y_predict,y_test)

0.9982121205893837

In [44]:
print(classification_report(y_predict,y_test))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00     36473
        True       0.99      1.00      1.00     12188

    accuracy                           1.00     48661
   macro avg       1.00      1.00      1.00     48661
weighted avg       1.00      1.00      1.00     48661



# testing the text

named entities cant be recognized

spelling mistakes are taken as hindi

shortcuts cant be predicted

mixture of hindi and normal text classification is sometimes unpredictable it depends on maximum no of words

it will not work for multi class classification

In [45]:
def testing_text(text):
    dummy=[]
    dummy.append(text)
    sequences=tokenizer.texts_to_sequences(dummy)
    print(sequences)
    padded=pad_sequences(sequences,maxlen=max_len)
    print(padded)
    y=model.predict(padded)
    print("predicted value :",y)
    y=y>0.5
    if y:
        print("text is hindi")
    else:
        print("text is english")

In [46]:
testing_text("hello morning how are you doing")

[[1794, 1034, 58, 293, 25, 782]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0 1794 1034   58  293   25  782]]
1/1 [==============================] - 0s 173ms/step
predicted value : [[2.8640998e-05]]
text is english


In [47]:
testing_text("kaise ho")

[[1, 1]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]]
1/1 [==============================] - 0s 31ms/step
predicted value : [[0.9994804]]
text is hindi


In [48]:
testing_text("bhaiyya kahn ho")

[[1, 1, 1]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]]
1/1 [==============================] - 0s 24ms/step
predicted value : [[0.99997854]]
text is hindi


In [49]:
testing_text("ap kya kar rahe ho")

[[2671, 1, 1, 1, 1]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0 2671    1    1    1    1]]
1/1 [==============================] - 0s 32ms/step
predicted value : [[0.9999991]]
text is hindi
